In [1]:
from __future__ import division
from __future__ import print_function
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from utils import accuracy
from models import iterativeGCN

In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sizhuang. Use `wandb login --relogin` to force relogin


True

In [3]:
sweep_config = {
    'method': 'random'
}

In [4]:
metric = {
    'name': 'accuracy',
    'goal': 'maximize'
}
sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'num_iterations': {
        'values':[2, 3]
    },
    'learning_rate': {
        'values': np.arange(0.0005, 0.02, 0.0005).tolist()
    },
    'smooth_fac': {
        'values': np.arange(0.1, 1, 0.05).tolist()
    },
    'hid_dim': {
        'values': [16, 32]
    },
    'weight_decay': {
        'values': [1e-4, 2e-4, 3e-4, 4e-4, 5e-4]
    },
    'num_runs': {
        'value': 5
    },
    'num_epochs': {
        'value': 1 #will change to 200
    },
    'dropout': {
        'value': 0.5
    }
}
sweep_config['parameters'] = parameters_dict

In [6]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'dropout': {'value': 0.5},
                'hid_dim': {'values': [16, 32]},
                'learning_rate': {'values': [0.0005,
                                             0.001,
                                             0.0015,
                                             0.002,
                                             0.0025,
                                             0.003,
                                             0.0035,
                                             0.004,
                                             0.0045000000000000005,
                                             0.005000000000000001,
                                             0.0055,
                                             0.006,
                                             0.006500000000000001,
                                             0.007000000000000001,
                                        

In [7]:
sweep_id = wandb.sweep(sweep_config, project="IterativeMethods")


Create sweep with ID: 0mqq9bfx
Sweep URL: https://wandb.ai/sizhuang/IterativeMethods/sweeps/0mqq9bfx


In [8]:
def build_model(config, input_dim, output_dim):
    model = iterativeGCN(input_dim=input_dim,
                            output_dim=output_dim,
                            hidden_dim=config.hid_dim,
                            num_train_iter=config.num_iterations,
                            smooth_fac=config.smooth_fac,
                            dropout=config.dropout)
    return model

def train_epoch(model, data, optimizer):
    model.train()
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    pred = output[data.train_mask].argmax(dim=1)
    acc = accuracy(pred, data.y[data.train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, acc

def validate_epoch(model, data):
    model.eval()
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.val_mask], data.y[data.val_mask])
    pred = output[data.val_mask].argmax(dim=1)
    acc = accuracy(pred, data.y[data.val_mask])
    return loss, acc

def test(model, data):
    model.eval()
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.test_mask], data.y[data.test_mask])
    pred = output[data.test_mask].argmax(dim=1)
    acc = accuracy(pred, data.y[data.test_mask])
    
    return loss, acc

def run_exp(config=None):
    with wandb.init(config=config):
        config = wandb.config

        dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
        data = dataset[0]
        input_dim = dataset.num_features
        output_dim = dataset.num_classes
        model = build_model(config, input_dim, output_dim)
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
        for epoch in range(config.num_epochs):
            loss_train, acc_train = train_epoch(model, data, optimizer)
            loss_val, acc_val = validate_epoch(model, data)
            wandb.log({
                'training_loss': loss_train,
                'training_accuracy': acc_train,
                'validation_loss': loss_val,
                'validation_accuracy': acc_val,
                "epoch": epoch
            })
        
        loss_test, acc_test = test(model, data)
        wandb.log({
                'test_loss': loss_test,
                'test_accuracy': acc_test
        })


In [9]:
wandb.agent(sweep_id, run_exp, count=5)

wandb: Agent Starting Run: 9cde0sqs with config:
wandb: 	dropout: 0.5
wandb: 	hid_dim: 16
wandb: 	learning_rate: 0.015
wandb: 	num_epochs: 1
wandb: 	num_iterations: 3
wandb: 	num_runs: 5
wandb: 	smooth_fac: 0.40000000000000013
wandb: 	weight_decay: 0.0005
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁
training_loss,▁
validation_accuracy,▁
validation_loss,▁
epoch,0
test_accuracy,0.13
test_loss,1.95242
training_accuracy,0.14286


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 5o9anidx with config:
wandb: 	dropout: 0.5
wandb: 	hid_dim: 16
wandb: 	learning_rate: 0.0115
wandb: 	num_epochs: 1
wandb: 	num_iterations: 3
wandb: 	num_runs: 5
wandb: 	smooth_fac: 0.30000000000000004
wandb: 	weight_decay: 0.0002
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁
training_loss,▁
validation_accuracy,▁
validation_loss,▁
epoch,0
test_accuracy,0.149
test_loss,1.98316
training_accuracy,0.14286


wandb: Agent Starting Run: z90j8zte with config:
wandb: 	dropout: 0.5
wandb: 	hid_dim: 16
wandb: 	learning_rate: 0.0145
wandb: 	num_epochs: 1
wandb: 	num_iterations: 3
wandb: 	num_runs: 5
wandb: 	smooth_fac: 0.15000000000000002
wandb: 	weight_decay: 0.0004
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁
training_loss,▁
validation_accuracy,▁
validation_loss,▁
epoch,0
test_accuracy,0.319
test_loss,1.9036
training_accuracy,0.14286


wandb: Agent Starting Run: z5nb5tvw with config:
wandb: 	dropout: 0.5
wandb: 	hid_dim: 16
wandb: 	learning_rate: 0.002
wandb: 	num_epochs: 1
wandb: 	num_iterations: 2
wandb: 	num_runs: 5
wandb: 	smooth_fac: 0.7000000000000002
wandb: 	weight_decay: 0.0002
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁
training_loss,▁
validation_accuracy,▁
validation_loss,▁
epoch,0
test_accuracy,0.103
test_loss,1.9531
training_accuracy,0.14286


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: lguyozv6 with config:
wandb: 	dropout: 0.5
wandb: 	hid_dim: 16
wandb: 	learning_rate: 0.0045000000000000005
wandb: 	num_epochs: 1
wandb: 	num_iterations: 2
wandb: 	num_runs: 5
wandb: 	smooth_fac: 0.20000000000000004
wandb: 	weight_decay: 0.0002
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁
training_loss,▁
validation_accuracy,▁
validation_loss,▁
epoch,0
test_accuracy,0.064
test_loss,1.9408
training_accuracy,0.14286
